In [ ]:
import numpy as np
import torch
from function import *

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# number of nodes in each cluster.
n_units = [300, 300, 100, 100, 100, 100]

# number of clusters or labels
n_L = len(n_units)

# w_k in the objective function is defined by w_k = alpha ^ (k-1)
# exp_base below indicates alpha
# the value can be varied.
exp_base = 1

# learning rate
lr = 0.4

epochs = 10

# All probability parameter values (p2, q2, p3_1, p3_2, q3, ....)  can be adjusted
# Diagonals of B_2 are all p2, and off-diagonals are all q2
p2 = 0.01
q2 = 0.001

# Diagonals of B_3 is set to [p3_1, p3_1, p3_2, p3_2, p3_2, p3_2], and off-diagonals are all q3
p3_1 = 0.03
p3_2 = 0.06
p3 = [p3_1] * 2 + [p3_2] * 4
q3 = 0.006

# Diagonals of B_4 is set to [p4_1, p4_1, p4_2, p4_2, p4_2, p4_2], and off-diagonals are all q4
p4_1 = 0.04
p4_2 = 0.08
p4 = [p4_1] * 2 + [p4_2] * 4
q4 = 0.008

# Diagonals of B_5 is set to [p5_1, p5_1, p5_2, p5_2, p5_2, p5_2], and off-diagonals are all q5
p5_1 = 0.11
p5_2 = 0.17
p5 = [p5_1] * 2 + [p5_2] * 4
q5 = 0.017

# Create from B_2 to B_5
prob_mat2 = np.full((n_L, n_L), q2)
np.fill_diagonal(prob_mat2, p2)

prob_mat3 = np.full((n_L, n_L), q3)
np.fill_diagonal(prob_mat3, p3)

prob_mat4 = np.full((n_L, n_L), q4)
np.fill_diagonal(prob_mat4, p4)

prob_mat5 = np.full((n_L, n_L), q5)
np.fill_diagonal(prob_mat5, p5)

# Generate networks using SBTM up to 5 cliques
# G: Graph generated using networkx
# simplices: All cliques (simplices) in the network
# classes: A list of length n_L. classes[i] includes all node with label i
# labels: labels for all nodes from 1 to N where N is the number of nodes
G, simplices, classes, labels = generation_upto_5cliques(prob_mat2, prob_mat3, prob_mat4, prob_mat5, n_units)

# Initialize probability distribution of all nodes using the equilibrium measure
# initial_data: Nxl matrix where N is the number of nodes and l is the number of clusters or labels in the network
# initial_pred: argmax applied (to each row in initial_data) N-dimensionial vectors
# x_known: label-aware nodes and their one-hot label representations
# BP: prior information ratio
BP = 0.01
initial_data, initial_pred, x_known = initialization(G, classes, simplices, BP)

# final_P: predicted Nxl matrix
# pred: argmax applied (to each row in final_P) N-dimensionial vectors
final_P, pred = training(epochs, device, simplices, initial_data, x_known, lr, exp_base)